# 패키지

In [1]:
import os
import warnings
warnings.filterwarnings("ignore")
import matplotlib.pyplot as plt
import pickle
import numpy as np
import pandas as pd
from datetime import datetime
from eunjeon import Mecab
from tqdm.notebook import tqdm
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import SimpleRNN, Embedding, Dense, LSTM, GRU

In [3]:
path = "Data/"
os.listdir(path)

['abuse_process_log_20220106.csv', 'apt_review_20220106.csv']

# 1. 데이터 로딩

### 리뷰, 처리내역 로그

In [4]:
data_hgnn = pd.read_csv(path + "apt_review_20220106.csv", encoding="UTF-8")
data_log = pd.read_csv(path + 'abuse_process_log_20220106.csv', encoding="UTF-8")

### 리뷰 데이터 전처리

In [5]:
# 처리내역 데이터 가공
data_log = data_log[(data_log["type"]==1)&(data_log["comment_id"].isnull())] # Type == 1 : 아파트 리뷰
data_log = data_log[~((data_log["reason"].str.contains("무효"))|(data_log["reason"].str.contains("복원")))]
data_log = data_log[["review_id", "reason"]].dropna().drop_duplicates("review_id")

# 병합
data_hgnn = data_hgnn[["id", "content", "is_blocked", "is_deleted", "is_blinded"]]
data = pd.merge(data_hgnn, data_log.rename(columns={"review_id":"id"}), on="id", how="left")

# 필터링
data = data[~((data["is_deleted"]==1)&(data["is_blocked"]!=1)&(data["is_blinded"]!=1))] # 자진삭제한 데이터
data = data[~data["content"].isnull()] # 내용이 기재되지 않은 데이터
data = data[data["content"].str.len() > 20] # 최소 20자 이상 리뷰

# 스팸 기준: is_blocked = 1이거나, 처리사유가 기재된 데이터
data["is_spam"] = np.where((data["is_blocked"]==1)|(~data["reason"].isnull()), 1, 0)

# 각 문장별, 1회 이상 스팸 처리된 문장은 is_spam에서 1, 나머지는 0으로 처리 (완전 중복문장 제거)
data_input = data.groupby("content").sum()[["is_spam"]].reset_index()
data_input["is_spam"] = np.where(data_input["is_spam"] > 0, 1, 0)

### 키워드 형태소 분리

In [7]:
tagger = Mecab()
morphs = list()
for i in tqdm(range(len(data_input))):
    corpus = data_input["content"].iloc[i]
    try:
        keywords = tagger.morphs(corpus.upper())
        morphs.append([keywords])
        
    except KeyboardInterrupt:
        break
    
    except:
        morphs.append([])

### 형태소 단위로 띄어써서 재구축

In [8]:
data_input = pd.concat([pd.DataFrame(morphs)[0], data_input["is_spam"].reset_index(drop=True)], axis=1).rename(columns={0:"content"}).dropna()
data_input["content"] = data_input["content"].apply(lambda row: " ".join(row))

### 후처리 (`zero with space` 삭제)

In [9]:
data_input["content"] = data_input["content"].str.replace("\u200b","")

In [11]:
data_input

,content,is_spam
0," 지식 이 일정 수준 이상 이 며 , 다양 한 활동 경험 이 있 으신 분 들 이 ...",0
1,"목포 , 내항 , 남항 북항 서남권 경제 특화 항만 으로 개발 추진 HTTP : /...",0
2,박상돈 천안시장 특별 대담 … 조정 지역 해제 건 의 시사 HTTPS : / / B...,0
3,박상돈 천안시장 특별 대담 … 조정 지역 해제 건 의 시사 HTTP : / / WW...,0
4,"서연 이 음 터 도서관 화성시 장지동 115 - 1 대지면적 : 3 , 250 M ...",0
...,...,...
1993209,🤥 🤐 🤐 😯 🤢 😯 👹 😶 👿 😈 🤫 🤕🤢🎃🤕😺🤢🤢🐧🐴🙈🦇🐝🐵🦗🐸🪲🦋🙉🐮🐜🦋🐦🐛🙉🐜,1
1993210,🥈 🎇 🎎 🎁 🎳 🏉 🏆 🧧🏉🎐🏈🏏🏓🏑🏈🏒🧧🏈🎖🥈🎫🥈🥈🥎🥈🏀🥈🏅🥈🎑🥈🎖 ⚾ ️ 🏉 ...,1
1993211,🥥🥦🌽🥔🥖🧈🍗🍟🍕🦴🦴🥞🤿🤺🥅🤾🏽 ‍♂ ️ 🤼 🪂 🧬 🛁 🩸 🪣 🩸 🧸 🪆 🕳 🛋 🇬...,1
1993212,🥬 🥦 🫐 🥭 🍌 🍋🍊🍐🫑🥐🥨🏤🛖🏞🏞🧭🌐🧱🏢🏬🏯🏏🥎🎑🧨🎎🥉🎾🏏🏏,1


# 2. 학습용 데이터 세팅

In [10]:
# X, y 세팅
X = data_input["content"]
y = data_input["is_spam"]

# 학습-테스트 데이터 분리
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0, stratify=y)

In [10]:
# 리뷰 문장 토큰화
tokenizer = Tokenizer()
tokenizer.fit_on_texts(X_train)
X_train_encoded = tokenizer.texts_to_sequences(X_train)

# 단어집합 총 크기
word_to_index = tokenizer.word_index
vocab_size = len(word_to_index) + 1
print(f"단어집합 사이즈: {vocab_size}")
print(f'리뷰 시퀀스 최대 길이: {max(len(l) for l in X_train_encoded)}')
print(f'리뷰 시퀀스 평균 길이: {(sum(map(len, X_train_encoded))/len(X_train_encoded))}')

# 훈련 데이터 패딩
max_len = 512
X_train_padded = pad_sequences(X_train_encoded, maxlen = max_len)
print(f"훈련 데이터의 크기(shape): {X_train_padded.shape}")

단어집합 사이즈: 156192
리뷰 시퀀스 최대 길이: 22771
리뷰 시퀀스 평균 길이: 44.24576233452425
훈련 데이터의 크기(shape): (1594569, 512)


# 3. 모형 학습

### 모형 구축

In [11]:
model = Sequential()
model.add(Embedding(vocab_size, 64))
model.add(LSTM(32))
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer='Adam', loss='binary_crossentropy', metrics=['acc'])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, None, 64)          9996288   
_________________________________________________________________
lstm (LSTM)                  (None, 32)                12416     
_________________________________________________________________
dense (Dense)                (None, 1)                 33        
Total params: 10,008,737
Trainable params: 10,008,737
Non-trainable params: 0
_________________________________________________________________


### 학습

In [12]:
history = model.fit(X_train_padded, y_train, epochs=2, batch_size=512, validation_split=0.25)

Epoch 1/2
2336/2336 [==============================] - 239s 102ms/step - loss: 0.0996 - acc: 0.9685 - val_loss: 0.0874 - val_acc: 0.9704
Epoch 2/2
2336/2336 [==============================] - 239s 103ms/step - loss: 0.0789 - acc: 0.9722 - val_loss: 0.0854 - val_acc: 0.9710


### 테스트 정확도 확인

In [13]:
X_test_encoded = tokenizer.texts_to_sequences(X_test)
X_test_padded = pad_sequences(X_test_encoded, maxlen = max_len)
print(f"테스트 정확도: {model.evaluate(X_test_padded, y_test)[1]}")

12458/12458 [==============================] - 172s 14ms/step - loss: 0.0849 - acc: 0.9713
테스트 정확도: 0.9712600111961365


### 학습된 모델 저장

In [14]:
model.save(f"Models/Spam_Classifier.h5")
with open(f'Models/Tokenizer.pickle', 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)

# 4. 테스트

### 모형 로딩

In [ ]:
model = load_model(f"Models/Spam_Classifier.h5")
with open(f'Models/Tokenizer.pickle', 'rb') as handle:
    tokenizer = pickle.load(handle)

### 예측

In [2]:
def separated(array):
    return [" ".join(tagger.morphs(array[i])) for i in range(len(array))]

In [15]:
def predict(array):
    array = separated(list(array))
    array = tokenizer.texts_to_sequences(array)
    array = pad_sequences(array, maxlen = 512)
    return model.predict(array)

In [26]:
# 테스트 데이터로 확률 예측
data_test = pd.concat([X_test, y_test], axis=1)
prob = predict(data_test["content"])

In [78]:
# 형태소 분리 전 원본 데이터 복구
data_test = pd.merge(data_test.reset_index().rename(columns={"content":"content_edit"}), 
                     data.groupby("content").sum()[["is_spam"]].reset_index().reset_index()[["index", "content"]], 
                     on="index", how="left")

In [83]:
# 예측 데이터 정리
result = pd.concat([data_test, pd.DataFrame(prob, columns=["prob"])], axis=1)[["content", "content_edit", "is_spam", "prob"]]
result["verdict"] = np.where(result["prob"] > 0.5, 1, 0)
result["prob"] *= 100

### 테스트 결과 저장

In [85]:
result.to_excel("리뷰별 스팸성 리뷰 예측결과.xlsx", index=False)